In [3]:
!pip install pvlib
from typing import Tuple
from datetime import datetime

import pvlib


def solar(
    E_ghi: float, T: float, R: float, t: datetime, *,
    A_t: float, A_a: float, P_lon_lat: Tuple[float, float],
    G_s: float, T_opt: float, E_std: float, T_std: float, P: float
) -> float:
    """
    Predict the photovoltaic power production.

    :param E_ghi: global horizontal irradiance
    :param T: temperature
    :param R: ground albedo (reflectance of the ground around)
    :param t: calendar time with precision up to seconds, in UTC
    :param A_t: tilt angle
    :param A_a: azimuth angle
    :param P_lon_lat: longitude and latitude of the location
    :param G_s: linear coefficient of efficiency decrease
    :param T_opt: operational temerature of installation
    :param E_std: standard irradiance
    :param T_std: standard temperature
    :param P: capacity of installation

    :return: The power prediction of installation in Watts.
    """
    # Compute the Solar Zenith Angle and Solar Azimuth Angle
    sza_saa = pvlib.solarposition.get_solarposition(t,
                                                    P_lon_lat[1], P_lon_lat[0])
    # Extract SZA and SAA
    A_sza, A_saa = sza_saa['zenith'], sza_saa['azimuth']
    # Compute DHI and DNI (Direct Normal Irradiance and
    # Diffuse Horizontal Irradiance) using Erbs model
    dhi_dni = pvlib.irradiance.erbs(E_ghi, A_sza, t)
    # Extract DHI and DNI
    E_dhi, E_dni = dhi_dni['dhi'], dhi_dni['dni']
    # Computes the total irradiance
    irr = pvlib.irradiance.get_total_irradiance(A_t, A_a, A_sza, A_saa,
                                                E_dni, E_ghi, E_dhi, albedo=R)
    # Extract total irradiance
    E_irr = irr['poa_global'].to_numpy().squeeze()

    # Normalized temperature
    T_n = T + E_irr * (T_opt - 20.0) / 800.0

    # Predict the power:
    return (E_irr / E_std) * (1.0 + G_s) * (T_n - T_std) * P


# EXAMPLE
power = solar(E_ghi=800.0, T=23.5, R=0.25, t=datetime(2020, 5, 12, 10, 30),
              A_t=30.0, A_a=30.0, P_lon_lat=(-0.12795, 51.50774),
              G_s=-0.004, T_opt=47.0, E_std=1000.0, T_std=43.0, P=1000.0)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


696.1568109794828

In [4]:
power

696.1568109794828


###Reference
### William F. Holmgren, Clifford W. Hansen, and Mark A. Mikofski. “pvlib python: a python package for modeling solar energy systems.” Journal of Open Source Software, 3(29), 884, (2018). https://doi.org/10.21105/joss.00884